<a href="https://colab.research.google.com/github/MicroPyscho/MicroPyscho/blob/main/ANN_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow

In [2]:
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import torch
from google.colab import drive
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import PowerTransformer
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from torch import nn

Load dataset and replace all +ve and -ve infinite values with NaN

In [3]:
star_df = pd.read_csv("/content/drive/MyDrive/Star_df dataset").replace([np.inf, -np.inf], np.nan)

if "Unnamed: 0" in star_df.columns:
  star_df.drop(columns =["Unnamed: 0"], inplace=True)

<ipython-input-3-4eea08218dbb>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  star_df = pd.read_csv("/content/drive/MyDrive/Star_df dataset").replace([np.inf, -np.inf], np.nan)


In [4]:
var_sum = ['_CGVSDP046', '_CGVSDP047', '_CGVSDP048', '_CGVSDP049',
                    '_CGVSDP050', '_CGVSDP053', '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058']
star_df['SDG5_9&12_17'] = star_df[var_sum].sum(axis=1)
star_df.drop(columns=var_sum, inplace=True)

In [5]:
var_sum2 = ['_ENERDP062', '_ENERDP063', '_ENERDP068', "Target", "CCIndexd1", "SUS_report", "ENV_committee"]
star_df['waste_redctn_efforts'] = (star_df[var_sum2].sum(axis=1) / 7).clip(0, 1)
star_df.drop(columns=var_sum2, inplace=True)


In [6]:
star_df['_ENRRO04V'] = (star_df['_ENRRO04V'] - star_df['_ENRRO04V'].mean()) / star_df['_ENRRO04V'].std()
var_sum3 = ["_ENPIDP066", "_ENPIDP069", "_ENRRO04V", "EcoInn_score", "_ENERO24V"]
star_df['clean_tech_efrts'] = (star_df[var_sum3].sum(axis=1) / 5).clip(0, 1)
star_df.drop(columns=var_sum3, inplace=True)

In [7]:
var_sum4 = ["_SOCODP011", "_SOCOO10V", "CrisisMgt"]
star_df['anomaly_reportng_efrts'] = (star_df[var_sum4].sum(axis=1) / 3).clip(0, 1)
star_df.drop(columns=var_sum4, inplace=True)

In [8]:
star_df['_SODODP017'] = (star_df['_SODODP017'] - star_df['_SODODP017'].mean()) / star_df['_SODODP017'].std()
star_df['_SODODP019'] = (star_df['_SODODP019'] - star_df['_SODODP019'].mean()) / star_df['_SODODP019'].std()
var_sum5 = ["_SODODP019", "_SODODP017"]
star_df['women_rpresntn'] = (star_df[var_sum5].sum(axis=1) / 2).clip(0, 1)
star_df.drop(columns=var_sum5, inplace=True)

In [9]:
var_sum6= ['climatevul_capacity', 'climatevul_exposure', 'climatevul_sensitivity',
                    'climate_vul', 'climate_economic', 'climate_governance',
                    'climate_readiness', 'climate_social']
star_df['climate_preparedness'] = star_df[var_sum6].sum(axis=1)
star_df.drop(columns=var_sum6, inplace=True)

In [10]:
var_avg = ['CO2eGDP', 'CO2eGDP_PPP2017', 'CO2eGDP_PPP']
star_df['avgCO2_GDP'] = star_df[var_avg].mean(axis=1)
star_df.drop(columns=var_avg, inplace=True)

In [11]:
star_df['bal_of_tradeGDP%'] = star_df['ExportsGDP'] - star_df['ImportsGDP']
var_std = ['GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', "MktCapGDP", "GovtDebtGDP", "bal_of_tradeGDP%", "TradeGDP"]
for var in var_std:
    star_df[f'{var}_standardized'] = (star_df[var] - star_df[var].mean()) / star_df[var].std()
star_df['gdp_prfmnce'] = star_df[[f'{var}_standardized' for var in var_std]].sum(axis=1)
star_df['gdp_prfmnce_rank'] = star_df['gdp_prfmnce'].rank(pct=True) * 100
star_df.drop(columns=var_std, inplace=True)

In [12]:
income_class_map = {'High income': 4, 'Upper middle income': 3, 'Lower middle income': 2, 'Low income': 1}
star_df['incme_class'] = star_df['ficincome'].map(income_class_map)
var_std2 = ['var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score', 'incme_class', "gain"]
for var in var_std2:
  star_df[f'{var}_standardized'] = (star_df[var] - star_df[var].mean()) / star_df[var].std()
star_df['eco_soc_gov_fctrs'] = star_df[[f'{var}_standardized' for var in var_std2]].sum(axis=1)
star_df['eco_soc_gov_fctrs'] = (star_df['eco_soc_gov_fctrs'] - star_df['eco_soc_gov_fctrs'].min()) / (star_df['eco_soc_gov_fctrs'].max() - star_df['eco_soc_gov_fctrs'].min()) * 100
star_df.drop(columns=var_std2, inplace=True)

In [13]:
star_df['dvsrty&inclsn_score'] = (star_df['_TRDIRDS'] + star_df['_TRDIRIS']) / 2

star_df['total_emissns'] = star_df['_ENERDP023'] + star_df['_ENERDP123'] + star_df["_ENERO03V"]
star_df['Net_FDI_GDP'] = star_df['FDI_INFGDP'] - star_df['FDI_OUTGDP']

star_df['avg_EPS'] = (star_df['_EPS'] + star_df['_EPS2MN']) / 2
star_df['DCred2PrvSctr_GDP'] = (star_df['DCredPvtGDP'] - star_df['DCredPvtGDP'].mean()) / star_df['DCredPvtGDP'].std()
star_df['agg_net_cash_flow'] = star_df['_WC04860'] + star_df['_WC04870'] + star_df['_WC04890']
star_df['operating_profit'] = star_df['_WC01250'] - star_df['_WC01249']
star_df["abs_sales_growth"] = star_df['sgrowth'] - star_df['sicsgrowth']
star_df.rename(columns={'_WC03151': 'working_capital'}, inplace=True)
star_df.rename(columns={"_WC01001": "net_revenue"}, inplace=True)
star_df.rename(columns={"_WC01201": "R&D"}, inplace=True)
star_df.rename(columns={"_WC03501": "common_equity"}, inplace=True)
star_df.rename(columns={"_WC03998": "total_capital"}, inplace=True)
star_df.rename(columns={"_WC08106": "current_ratio"}, inplace=True)
star_df.rename(columns={"_WC08126": "invntry_hold_days"}, inplace=True)
star_df.rename(columns={"_WC08366": "net_prfit_margin"}, inplace=True)
star_df.rename(columns={"roe": "retrn_on_equity"}, inplace=True)
star_df.rename(columns={"_CGBSO19": "board_gndr_div%"}, inplace=True)
star_df.rename(columns={"_CGSCORE": "govnce_pilr_score"}, inplace=True)
star_df.rename(columns={"_ENERDP024": "dirct_cemissions"}, inplace=True)
star_df.rename(columns={"_ENERDP025": "indirct_cemissions"}, inplace=True)
star_df.rename(columns={"_ENERO27V": "scope1"}, inplace=True)
star_df.rename(columns={"_ENERO28V": "scope2"}, inplace=True)
star_df.rename(columns={"_ENRRDP033": "total_enrgy_use"}, inplace=True)
star_df.rename(columns={"_ENRRDP046": "renwbl_enrgy_use"}, inplace=True)
star_df.rename(columns={"_ENSCORE": "envrmt_pilr_score"}, inplace=True)
star_df.rename(columns={"_TRESGCS": "ESG_score"}, inplace=True)
star_df.rename(columns={"_TRESGCGVSS": "CSR_score"}, inplace=True)
star_df.rename(columns={"_TRESGENERS": "emissn_score"}, inplace=True)
star_df.rename(columns={"INTSpread": "Intrst_rate_spread"}, inplace=True)
star_df.rename(columns={"pop_score": "population"}, inplace=True)
star_df.rename(columns={"_MV": "market_value"}, inplace=True)
star_df.rename(columns={"_PI": "price_index"}, inplace=True)
star_df.rename(columns={"_RI": "return_index"}, inplace=True)
star_df.rename(columns={"SIC": "SIC_code"}, inplace=True)
star_df.rename(columns={"_ENERDP0161": "Emission_objtv"}, inplace=True)
star_df.rename(columns={"_ENERO03V": "Grenhous_emissn"}, inplace=True)
star_df.rename(columns={"_ENERO26V": "CO2Emissn_%+-"}, inplace=True)
star_df.rename(columns={"_CGSRDP049": "prvt0_vs_govtowned1"}, inplace=True)



#star_df['working_capital'] = star_df['_WC02201'] - star_df['_WC02999']

In [14]:
list_A = [
    'DSCD', 'ticker', 'exchange', 'WC06011', 'WC07040', '_EPS', '_EPS2MN', '_MVC',
    '_P', '_WC01051', '_WC01084', '_WC01249', '_WC01100', '_WC01250', '_WC01401',
    '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999',
    '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551',
    '_WC04601', '_WC04860', '_WC04870', '_WC04890', '_WC05201', '_WC05476',
    '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366',
    '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales',
    'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta',
    'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep',
    'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales',
    'dcashna', 'sgrowth', 'sicsgrowth', 'nwccta', 'divta', 'tri', 'divchg', 'lppe',
    'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta',
    'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales',
    'lcash2ta', 'lempgrowth', 'lcapexrdta', 'lcash2sales', 'lcash2na', 'wwindex',
    'kzindex', 'saindex', 'mwail', 'zscore', 'zscoree', '_CGSRDP049', '_CGVSDP046',
    '_CGVSDP047', '_CGVSDP048', '_CGVSDP049', '_CGVSDP050', '_CGVSDP053',
    '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058',
    '_ENERDP033', '_ENERDP062', '_ENERO32V', '_SOCODP037', '_SODODP0151',
    '_SOEQO08V', '_TRDIRDs', '_TRDIRIS', '_TRESGS', '_TRESGENPIS', 'TCE', 'DCE',
    'INDCE', 'logTCE', 'logDCE', 'logINDCE', 'logEMScore', 'TCETA', 'DCETA',
    'ENVInitiatives', 'logENVInvScore', 'EnergyUse', 'RenEnergyUse',
    'EnergyUseSale', 'logEnergyUse', 'logResUse_score', 'logMgtScore',
    'logSHScore', 'logCSRScore', 'logCMScore', 'logHRScore', 'logPRScore',
    'GOVScore', 'SOSScore', 'ESG_Comp', 'MgersFem', 'BODDIV', 'EXEDIV',
    'SnrExecCompTA', 'SnrExecCompSale', 'CrisisMgt', 'EmissionsTrdg', 'SDG13',
    'Climate_gov', 'ENV_Prod', 'ENV_Asset', 'Eco_Prod', 'RespUse_Prod',
    'CleanTech', 'EcoInn_score', 'ENV_train', 'MgtTrg', 'ESGC', 'CO2eGDP',
    'CO2eGDP_PPP2017', 'CO2eGDP_PPP', 'FDI_OUTGDP', 'FDI_INFGDP', 'DCredPvtGDP',
    'GDP_Cur', 'GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', 'MktCapGDP',
    'GovtDebtGDP', 'bal_of_tradeGDP%', 'INFL_conpx', 'INTPayExp', 'INTPayRev',
    'RealINT', 'gdp_score', 'var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score',
    'incme_class', 'gain', 'ficincome', 'climatevul_capacity', 'climatevul_exposure',
    'climatevul_sensitivity', 'climate_vul', 'climate_economic', 'climate_governance',
    'climate_readiness', 'climate_social'
]

list_B = [
    '_CGSRDP049', '_CGVSDP047', '_CGVSDP048', '_CGVSDP049',
    '_CGVSDP050', '_CGVSDP053', '_CGVSDP054', '_CGVSDP055', '_CGVSDP056', '_CGVSDP057', '_CGVSDP058', '_ENERDP062', '_ENERDP063', '_ENERDP068', "Target", "CCIndexd1", "SUS_report", "ENV_committee", "_ENPIDP066", "_ENPIDP069", "_ENRRO04V", "EcoInn_score", "_SOCODP011", "_SOCOO10V", "CrisisMgt", "_SODODP019", "_SODODP017", 'climatevul_capacity', 'climatevul_exposure', 'climatevul_sensitivity',
                    'climate_vul', 'climate_economic', 'climate_governance',
                    'climate_readiness', 'climate_social', 'CO2eGDP', 'CO2eGDP_PPP2017', 'CO2eGDP_PPP',  'GDP_K2015', 'GDPg', 'GDP_percap2015', 'GDPg_percap', "MktCapGDP", "GovtDebtGDP", "bal_of_tradeGDP%", "TradeGDP", 'var', 'pvr', 'ger', 'rqr', 'rlr', 'ccr', 'hdi_score', 'incme_class', "gain"
]

# Remove duplicates from list_B in list_A
list_A = [variable for variable in list_A if variable not in list_B]

print(list_A)


['DSCD', 'ticker', 'exchange', 'WC06011', 'WC07040', '_EPS', '_EPS2MN', '_MVC', '_P', '_WC01051', '_WC01084', '_WC01249', '_WC01100', '_WC01250', '_WC01401', '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999', '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551', '_WC04601', '_WC04860', '_WC04870', '_WC04890', '_WC05201', '_WC05476', '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366', '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales', 'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta', 'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep', 'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales', 'dcashna', 'sgrowth', 'sicsgrowth', 'nwccta', 'divta', 'tri', 'divchg', 'lppe', 'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta', 'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales', 'lcas

In [15]:
col_drop = ['DSCD', 'ticker', "_ENRRDP0341", "_ENERDP023", "Comp_CS", "Whistleblower", "ENV_RIC", "InsideDealC", "AcctC", "BribeCorrFraudC", "WagesWorkC", "RepsMarketC", "CustomerC", "_ENERDP123", "_ENERO24V", "_ENERO25V", "logmktcap", "size", "arna", "apsale", "cfw", "_TRDIRDS", "countryname", 'exchange', 'WC06011', 'WC07040', '_EPS', '_EPS2MN', '_MVC', '_P', '_WC01051', '_WC01084', '_WC01249', '_WC01100', '_WC01250', '_WC01401', '_WC02001', '_WC02051', '_WC02101', '_WC02201', '_WC02501', '_WC02999', '_WC03101', '_WC03255', '_WC03351', '_WC03480', '_WC03995', '_WC04551', '_WC04601', '_WC04860', '_WC04870', '_WC04890', '_WC05201', '_WC05476', '_WC05502', '_WC08101', '_WC08321', '_WC08301', '_WC08316', '_WC08366', '_WC09504', '_WC18193', '_WC18199', '_WC18198', 'ffi5', 'cashna', 'cash2sales', 'ppe', 'Size', 'logsales', 'logppe', 'firmage37', 'logage', 'tlta', 'tdta', 'tdmv', 'clta', 'cata', 'apta', 'arta', 'arsale', 'apcogs', 'nettcsale', 'dep', 'roa', 'ros', 'ebitta', 'tobinqe', 'mb', 'capexppe', 'dcashta', 'dcashsales', 'dcashna', 'sgrowth', 'sicsgrowth', 'nwccta', 'divta', 'tri', 'divchg', 'lppe', 'lsize', 'llogsales', 'lcfw', 'lroa', 'lroe', 'ltobinq', 'lcapexppe', 'larta', 'lapta', 'llogemp', 'ldep', 'llogage', 'llogage2', 'lcashna', 'lcashsales', 'lcash2ta', 'lempgrowth', 'lcapexrdta', 'lcash2sales', 'lcash2na', 'wwindex', 'kzindex', 'saindex', 'mwail', 'zscore', 'zscoree', '_CGVSDP046', '_ENERDP033', '_ENERO32V', '_SOCODP037', '_SODODP0151', '_SOEQO08V', '_TRDIRDs', '_TRDIRIS', '_TRESGS', '_TRESGENPIS', 'TCE', 'DCE', 'INDCE', 'logTCE', 'logDCE', 'logINDCE', 'logEMScore', 'TCETA', 'DCETA', 'ENVInitiatives', 'logENVInvScore', 'EnergyUse', 'RenEnergyUse', 'EnergyUseSale', 'logEnergyUse', 'logResUse_score', 'logMgtScore', 'logSHScore', 'logCSRScore', 'logCMScore', 'logHRScore', 'logPRScore', 'GOVScore', 'SOSScore', 'ESG_Comp', 'MgersFem', 'BODDIV', 'EXEDIV', 'SnrExecCompTA', 'SnrExecCompSale', 'EmissionsTrdg', 'SDG13', 'Climate_gov', 'ENV_Prod', 'ENV_Asset', 'Eco_Prod', 'RespUse_Prod', 'CleanTech', 'ENV_train', 'MgtTrg', 'ESGC', 'FDI_OUTGDP', 'FDI_INFGDP', 'DCredPvtGDP', 'GDP_Cur', 'INFL_conpx', 'INTPayExp', 'INTPayRev', 'RealINT', 'gdp_score', 'ficincome']
col_drop = [variable.strip() for variable in col_drop]
# Drop the specified columns from the dataset
star_df.drop(columns=col_drop, inplace=True, errors='ignore')

# Check the updated dataset
print(star_df.head())

           ISIN  ICBIC               country  startyear  INCORP  year  \
0  AEE001901017     35  United Arab Emirates     2017.0  2003.0  2020   
1  AEN000401010     50  United Arab Emirates     2003.0  1979.0  2020   
2  AES000401019     50  United Arab Emirates     2005.0  1976.0  2020   
3  AEN000901019     50  United Arab Emirates     2005.0  1968.0  2020   
4  AEG000101010     50  United Arab Emirates     2003.0  1977.0  2020   

   market_value  price_index  return_index  net_revenue  ...  \
0      11000.00         47.8         53.83    9758283.0  ...   
1       1502.50        114.5        239.16    3776232.0  ...   
2        462.27         16.3         41.61     431627.0  ...   
3        818.06          9.4         20.15     183087.0  ...   
4        344.45         44.6         92.12     345879.0  ...   

   gain_standardized  eco_soc_gov_fctrs  dvsrty&inclsn_score  total_emissns  \
0           0.051478          76.295645                  9.5   3.608878e+06   
1           0.0514

In [16]:
star_df = star_df.loc[:, ~star_df.columns.str.endswith('_standardized')]

In [17]:
for col_name, count in star_df.count().items():
    print(f"{col_name}: {count}")

total_variables = len(star_df.columns)
print(f"Total number of variables: {total_variables}")


ISIN: 88234
ICBIC: 88234
country: 88234
startyear: 88234
INCORP: 88234
year: 88234
market_value: 88234
price_index: 88234
return_index: 88234
net_revenue: 88234
R&D: 88234
working_capital: 88234
common_equity: 88234
total_capital: 88234
current_ratio: 88234
invntry_hold_days: 88234
net_prfit_margin: 88234
SIC_code: 88234
ffi30: 88234
age: 88234
retrn_on_equity: 88234
tobinq: 88234
mktvalue: 88234
empgrowth: 88234
board_gndr_div%: 88234
govnce_pilr_score: 88234
prvt0_vs_govtowned1: 88234
Emission_objtv: 88234
dirct_cemissions: 88234
indirct_cemissions: 88234
Grenhous_emissn: 88234
CO2Emissn_%+-: 88234
scope1: 88234
scope2: 88234
total_enrgy_use: 88234
renwbl_enrgy_use: 88234
envrmt_pilr_score: 88234
CSR_score: 88211
ESG_score: 88211
emissn_score: 88211
logENVScore: 88211
Scope3: 88211
CCIndex: 88211
Climate_incentive: 88211
INFL_GDPdeflator: 88234
Intrst_rate_spread: 88109
ImportsGDP: 88234
ExportsGDP: 88234
region: 88234
population: 88234
ai_ranking: 88234
ai_score: 88234
ai_govt: 8823

In [18]:
columns_with_nan = ['CSR_score', 'ESG_score', 'emissn_score', 'logENVScore', 'Scope3', 'CCIndex', 'Climate_incentive', 'population', 'ai_ranking', 'ai_score', 'ai_govt', 'ai_tech', 'ai_infr', 'INFL_GDPdeflator', 'Intrst_rate_spread', 'avgCO2_GDP']
star_df[columns_with_nan] = star_df[columns_with_nan].round()
#star_df[columns_with_nan] = star_df[columns_with_nan].astype('int64')

In [19]:
float_cols = star_df.select_dtypes(include=['float64']).columns

# Convert float64 columns to int64
star_df[float_cols] = star_df[float_cols].astype('int64', errors='ignore')

In [20]:
float64_columns = ['CSR_score', 'ESG_score', 'emissn_score', 'logENVScore',
                   'Scope3', 'CCIndex', 'Climate_incentive', 'Intrst_rate_spread', 'avgCO2_GDP']

for col in float64_columns:
    star_df[col] = star_df[col].fillna(0).round().astype('int64')

In [21]:
print("Type of star_df:", type(star_df))

# Check the structure of star_df
print("\nStructure of star_df:")
star_df.info()


Type of star_df: <class 'pandas.core.frame.DataFrame'>

Structure of star_df:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88234 entries, 0 to 88233
Data columns (total 73 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   ISIN                    88234 non-null  object
 1   ICBIC                   88234 non-null  int64 
 2   country                 88234 non-null  object
 3   startyear               88234 non-null  int64 
 4   INCORP                  88234 non-null  int64 
 5   year                    88234 non-null  int64 
 6   market_value            88234 non-null  int64 
 7   price_index             88234 non-null  int64 
 8   return_index            88234 non-null  int64 
 9   net_revenue             88234 non-null  int64 
 10  R&D                     88234 non-null  int64 
 11  working_capital         88234 non-null  int64 
 12  common_equity           88234 non-null  int64 
 13  total_capital           8823

In [22]:
log_star_df = pd.DataFrame()

# Iterate through each column in star_df
for col in star_df.columns:
    # Check if the column contains numerical data
    if np.issubdtype(star_df[col].dtypes, np.number):  # Check if the data type is numerical
        # Apply log transformation and add it to log_star_df
        log_star_df[col] = np.log1p(star_df[col])
    else:
        # If the column contains non-numerical data, simply copy it to log_star_df
        log_star_df[col] = star_df[col]

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: invalid value encountered in log1p
  resul

In [23]:
log_star_df.to_csv('/content/drive/My Drive/log_star_df.csv', index=False)

Exclude all object dtype variable in preparation for ANN and CNN model

In [24]:
log_star_df = log_star_df.select_dtypes(exclude=['object'])

In [25]:
log_star_df.head()

,ICBIC,startyear,INCORP,year,market_value,price_index,return_index,net_revenue,R&D,working_capital,...,gdp_prfmnce_rank,eco_soc_gov_fctrs,dvsrty&inclsn_score,total_emissns,Net_FDI_GDP,avg_EPS,DCred2PrvSctr_GDP,agg_net_cash_flow,operating_profit,abs_sales_growth
0,3.583519,7.609862,7.602900,7.611348,9.305741,3.871201,3.988984,16.093627,0.0,13.807003,...,4.26268,4.343805,2.302585,15.098907,0.0,0.0,0.0,NaN,NaN,0.0
1,3.931826,7.602900,7.590852,7.611348,7.315218,4.744932,5.480639,15.144238,0.0,14.766859,...,4.26268,4.343805,2.302585,15.152275,0.0,0.0,0.0,NaN,NaN,0.0
2,3.931826,7.603898,7.589336,7.611348,6.137727,2.833213,3.737670,12.975319,0.0,12.075052,...,4.26268,4.343805,2.302585,15.152275,0.0,0.0,0.0,11.360788,NaN,0.0
3,3.931826,7.603898,7.585281,7.611348,6.708084,2.302585,3.044522,12.117722,0.0,9.858333,...,4.26268,4.343805,2.302585,15.152275,0.0,0.0,0.0,NaN,NaN,0.0
4,3.931826,7.602900,7.589842,7.611348,5.843544,3.806662,4.532599,12.753847,0.0,9.462887,...,4.26268,4.343805,2.302585,15.152275,0.0,0.0,0.0,10.359361,NaN,0.0


In [26]:
star_df.shape

(88234, 73)

Replace any residual "-" in any variable with the mean of the variable column

In [27]:
log_star_df.replace('-', pd.NA, inplace=True)

log_star_df = log_star_df.apply(pd.to_numeric, errors='coerce')

column_means = log_star_df.mean(skipna=True)
log_star_df.fillna(column_means, inplace=True)

log_star_df = np.log1p(log_star_df)

print("Updated DataFrame:")
print(log_star_df)

Updated DataFrame:
          ICBIC  startyear    INCORP      year  market_value  price_index  \
0      1.522467   2.152908  2.152099  2.153081      2.332701     1.583341   
1      1.595709   2.152099  2.150698  2.153081      2.118087     1.748318   
2      1.595709   2.152215  2.150521  2.153081      1.965394     1.343703   
3      1.595709   2.152215  2.150049  2.153081      2.042270     1.194706   
4      1.595709   2.152099  2.150580  2.153081      1.923306     1.570003   
...         ...        ...       ...       ...           ...          ...   
88229  1.574565   2.152678  2.151576  2.153138      1.489299     1.310506   
88230  1.574565   2.152562  2.151576  2.153196      1.880507     1.631370   
88231  1.614495   2.152562  2.151576  2.153196      2.266755     1.902174   
88232  1.574565   2.152678  2.151576  2.153196      1.489299     1.310506   
88233  1.574565   2.152562  2.151576  2.153196      2.036254     1.668474   

       return_index  net_revenue  R&D  working_capital  

/usr/local/lib/python3.10/dist-packages/pandas/core/internals/blocks.py:329: RuntimeWarning: invalid value encountered in log1p
  result = func(self.values, **kwargs)


Split the variables according to their dtypes and note them

In [28]:
obj_cols = log_star_df.select_dtypes(include=['object'])

int_cols = log_star_df.select_dtypes(include=["int", "int64", "float", "float64"])

print("Object Columns:")
print(obj_cols.columns)
print("\nInteger Columns:")
print(int_cols.columns)


Object Columns:
Index([], dtype='object')

Integer Columns:
Index(['ICBIC', 'startyear', 'INCORP', 'year', 'market_value', 'price_index',
       'return_index', 'net_revenue', 'R&D', 'working_capital',
       'common_equity', 'total_capital', 'current_ratio', 'invntry_hold_days',
       'net_prfit_margin', 'SIC_code', 'age', 'retrn_on_equity', 'tobinq',
       'mktvalue', 'empgrowth', 'board_gndr_div%', 'govnce_pilr_score',
       'prvt0_vs_govtowned1', 'Emission_objtv', 'dirct_cemissions',
       'indirct_cemissions', 'Grenhous_emissn', 'CO2Emissn_%+-', 'scope1',
       'scope2', 'total_enrgy_use', 'renwbl_enrgy_use', 'envrmt_pilr_score',
       'CSR_score', 'ESG_score', 'emissn_score', 'logENVScore', 'Scope3',
       'CCIndex', 'Climate_incentive', 'INFL_GDPdeflator',
       'Intrst_rate_spread', 'ImportsGDP', 'ExportsGDP', 'population',
       'ai_ranking', 'ai_score', 'ai_govt', 'ai_tech', 'ai_infr',
       'SDG5_9&12_17', 'waste_redctn_efforts', 'clean_tech_efrts',
       'anomaly

Do the same for all non-object dtype to note where they fall between "int", "int64", "float", "float64"

In [29]:
int_cols

,ICBIC,startyear,INCORP,year,market_value,price_index,return_index,net_revenue,R&D,working_capital,...,gdp_prfmnce_rank,eco_soc_gov_fctrs,dvsrty&inclsn_score,total_emissns,Net_FDI_GDP,avg_EPS,DCred2PrvSctr_GDP,agg_net_cash_flow,operating_profit,abs_sales_growth
0,1.522467,2.152908,2.152099,2.153081,2.332701,1.583341,1.607232,2.838706,0.0,2.695100,...,1.660640,1.675938,1.194706,2.778751,0.000000,0.000000,0.0,NaN,2.593551,0.0
1,1.595709,2.152099,2.150698,2.153081,2.118087,1.748318,1.868819,2.781563,0.0,2.757910,...,1.660640,1.675938,1.194706,2.782061,0.000000,0.000000,0.0,NaN,2.593551,0.0
2,1.595709,2.152215,2.150521,2.153081,1.965394,1.343703,1.555545,2.637293,0.0,2.570706,...,1.660640,1.675938,1.194706,2.782061,0.000000,0.000000,0.0,2.514529,2.593551,0.0
3,1.595709,2.152215,2.150049,2.153081,2.042270,1.194706,1.397363,2.573964,0.0,2.384933,...,1.660640,1.675938,1.194706,2.782061,0.000000,0.000000,0.0,NaN,2.593551,0.0
4,1.595709,2.152099,2.150580,2.153081,1.923306,1.570003,1.710658,2.621319,0.0,2.347834,...,1.660640,1.675938,1.194706,2.782061,0.000000,0.000000,0.0,2.430042,2.593551,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88229,1.574565,2.152678,2.151576,2.153138,1.489299,1.310506,1.310506,2.798488,0.0,2.681384,...,1.628131,1.408800,1.509918,2.756432,0.000000,0.000000,NaN,2.573208,2.593551,0.0
88230,1.574565,2.152562,2.151576,2.153196,1.880507,1.631370,1.640720,2.591741,0.0,NaN,...,1.560500,1.448822,1.509918,2.756432,0.526589,0.000000,NaN,2.427020,2.593551,0.0
88231,1.614495,2.152562,2.151576,2.153196,2.266755,1.902174,1.953989,2.800147,0.0,2.729401,...,1.560500,1.448822,1.509918,2.756432,0.526589,0.526589,NaN,2.599496,2.593551,0.0
88232,1.574565,2.152678,2.151576,2.153196,1.489299,1.310506,1.310506,2.803417,0.0,2.698546,...,1.560500,1.448822,1.509918,2.756432,0.526589,0.000000,NaN,2.660544,2.593551,0.0


#Random Forest Regression to predict Total Emission

In [30]:
# Assuming int_cols is your DataFrame with integer format total_emissions
A = int_cols.drop(columns=["total_emissns", "Emission_objtv", "dirct_cemissions", "emissn_score", "indirct_cemissions", "Grenhous_emissn", "CO2Emissn_%+-", "scope1", "scope2", "total_enrgy_use", "renwbl_enrgy_use", "envrmt_pilr_score", "Climate_incentive"])
B = int_cols["total_emissns"]

# Splitting the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3, random_state=42)

# Building the Random Forest regression model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=42)
rf_regressor.fit(A_train, B_train)

# Feature importances
feature_importances = rf_regressor.feature_importances_

# Getting the top 50 features
top_50_indices = feature_importances.argsort()[-20:][::-1]
top_50_features = A.columns[top_50_indices]
top_50_importances = feature_importances[top_50_indices]

# Plotting the top 50 features
plt.figure(figsize=(20, 12))
plt.barh(top_50_features, top_50_importances)
plt.xlabel('Feature Importance')
plt.title('Top 20 Features Predicting Total Emissions Using RF Regression')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features at the top
plt.savefig('/content/top_20_RFReg2.pdf')
plt.show()

# Predicting on the test set
B_pred = rf_regressor.predict(A_test)

# Creating a DataFrame to display actual and predicted values side by side
results_df = pd.DataFrame({'Actual': B_test, 'Predicted': B_pred})
print(results_df)

ValueError: Input X contains NaN.
RandomForestRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculating Mean Absolute Error (MAE)
mae = mean_absolute_error(B_test, B_pred)

# Calculating Mean Squared Error (MSE)
mse = mean_squared_error(B_test, B_pred)

# Calculating R-squared (R2) score
r2 = r2_score(B_test, B_pred)

# Printing the evaluation metrics
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R2) Score:", r2)


In [ ]:
# Saving evaluation metrics to a file
with open('/content/evaluation_metrics.txt', 'w') as f:
    f.write("Mean Absolute Error (MAE): {}\n".format(mae))
    f.write("Mean Squared Error (MSE): {}\n".format(mse))
    f.write("R-squared (R2) Score: {}\n".format(r2))

# Visualizing actual vs predicted values
plt.figure(figsize=(10, 6))
plt.scatter(B_test, B_pred)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted Values for Random Forest Regression')
plt.savefig('/content/actual_vs_predicted_RFReg.png')
plt.show()


#KNN Regression to predict AI Ranking

In [ ]:
# Split the data into features (A) and target variable (B)
A = int_cols.drop(columns=["total_emissns", "Emission_objtv", "emissn_score", "dirct_cemissions", "indirct_cemissions", "Grenhous_emissn", "CO2Emissn_%+-", "scope1", "scope2", "total_enrgy_use", "renwbl_enrgy_use", "envrmt_pilr_score", "Climate_incentive"])
B = int_cols["total_emissns"]

# Split the data into training and testing sets
A_train, A_test, B_train, B_test = train_test_split(A, B, test_size=0.3, random_state=42)

# Initialize the KNN regressor
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors

# Train the KNN model
knn_regressor.fit(A_train, B_train)

# Predictions
y_pred_train = knn_regressor.predict(A_train)
y_pred_test = knn_regressor.predict(A_test)

# Evaluate the model
# Mean squared error
train_mse = mean_squared_error(B_train, y_pred_train)
test_mse = mean_squared_error(B_test, y_pred_test)

# R-squared score
train_r2 = r2_score(B_train, y_pred_train)
test_r2 = r2_score(B_test, y_pred_test)

print(f'Train Mean Squared Error: {train_mse}')
print(f'Test Mean Squared Error: {test_mse}')
print(f'Train R-squared Score: {train_r2}')
print(f'Test R-squared Score: {test_r2}')


In [ ]:
from sklearn.neighbors import KNeighborsRegressor

# Weighted KNN
weighted_knn_regressor = KNeighborsRegressor(n_neighbors=5, weights='distance')  # Using 'distance' for weighted KNN

# Train the weighted KNN model
weighted_knn_regressor.fit(A_train, B_train)

# Predictions
y_pred_train_weighted = weighted_knn_regressor.predict(A_train)
y_pred_test_weighted = weighted_knn_regressor.predict(A_test)

# Evaluate the weighted KNN model
train_mse_weighted = mean_squared_error(B_train, y_pred_train_weighted)
test_mse_weighted = mean_squared_error(B_test, y_pred_test_weighted)
train_r2_weighted = r2_score(B_train, y_pred_train_weighted)
test_r2_weighted = r2_score(B_test, y_pred_test_weighted)

print("Weighted KNN:")
print(f'Train Mean Squared Error: {train_mse_weighted}')
print(f'Test Mean Squared Error: {test_mse_weighted}')
print(f'Train R-squared Score: {train_r2_weighted}')
print(f'Test R-squared Score: {test_r2_weighted}')

In [ ]:
from sklearn.kernel_ridge import KernelRidge

# Kernel Ridge Regression
krr_regressor = KernelRidge(kernel='rbf', gamma=0.1)  # Using radial basis function (RBF) kernel

# Train the Kernel Ridge Regression model
krr_regressor.fit(A_train, B_train)

# Predictions
y_pred_train_krr = krr_regressor.predict(A_train)
y_pred_test_krr = krr_regressor.predict(A_test)

# Evaluate the Kernel Ridge Regression model
train_mse_krr = mean_squared_error(B_train, y_pred_train_krr)
test_mse_krr = mean_squared_error(B_test, y_pred_test_krr)
train_r2_krr = r2_score(B_train, y_pred_train_krr)
test_r2_krr = r2_score(B_test, y_pred_test_krr)

print("\nKernel Ridge Regression:")
print(f'Train Mean Squared Error: {train_mse_krr}')
print(f'Test Mean Squared Error: {test_mse_krr}')
print(f'Train R-squared Score: {train_r2_krr}')
print(f'Test R-squared Score: {test_r2_krr}')

In [ ]:
metrics_filename = '/content/drive/My Drive/RF_eval_metrics.txt'
with open(metrics_filename, 'w') as f:
    f.write(f'Train Mean Squared Error: {train_mse}\n')
    f.write(f'Test Mean Squared Error: {test_mse}\n')
    f.write(f'Train R-squared Score: {train_r2}\n')
    f.write(f'Test R-squared Score: {test_r2}\n')

In [ ]:
# Create a DataFrame with actual and predicted rankings
compard_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Display the DataFrame
print(compard_df)

compard_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Reset index to have numerical index
compard_df.reset_index(drop=True, inplace=True)

# Plot the actual and predicted rankings
plt.figure(figsize=(10, 6))
compard_df.plot(kind='bar', figsize=(14, 8))
plt.title('Actual vs Predicted Rankings using KNN Regression')
plt.xlabel('Sample Index')
plt.ylabel('Ranking')
plt.xticks(rotation=45)
plt.tight_layout()
plt.savefig('/content/actual_predicteed_KNNReg.pdf')
plt.show()

Rank the top 50 important features for predicting Total Emissions

In [ ]:
# Define number of classes
num_classes = 5  # Adjust as needed

# Discretize the target variable into classes
B_train_discretized = np.digitize(B_train, np.arange(0, 101, 100/num_classes))
B_test_discretized = np.digitize(B_test, np.arange(0, 101, 100/num_classes))

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the model
rf_classifier.fit(A_train, B_train_discretized)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Get indices of top 20 features
top_20_indices = feature_importances.argsort()[-20:][::-1]

# Get top 20 feature names
top_20_features = A_train.columns[top_20_indices]

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x=feature_importances[top_20_indices], y=top_20_features)
plt.title('Top 20 Features Importance for Total Emission Using Random Forest')
plt.xlabel('Feature Importance Score')
plt.ylabel('Feature')
plt.xticks(rotation=45)  # Rotate x-axis labels for better readability
plt.tight_layout()  # Adjust layout to prevent clipping of labels
plt.savefig('/content/20feat_importance_RFReg.pdf')  # Save the plot
plt.show()  # Display the plot


In [ ]:
# Define number of classes
num_classes = 5  # Adjust as needed

# Discretize the target variable into classes
B_train_discretized = np.digitize(B_train, np.arange(0, 101, 100/num_classes))
B_test_discretized = np.digitize(B_test, np.arange(0, 101, 100/num_classes))

# Initialize the RandomForestClassifier
rf_classifier = RandomForestClassifier()

# Train the model
rf_classifier.fit(A_train, B_train_discretized)

# Get feature importances
feature_importances = rf_classifier.feature_importances_

# Get indices of top 50 features
top_50_indices = feature_importances.argsort()[-20:][::-1]

# Get top 50 feature names
top_50_features = A_train.columns[top_50_indices]

# Plot feature importance
plt.figure(figsize=(12, 8))
sns.barplot(x=feature_importances[top_50_indices], y=top_50_features)
plt.title('Top 20 Features Importance for Total Emission Using KNN')
plt.xlabel('Feature Importance Score')
plt.ylabel('Feature')
plt.savefig('/content/50feat_importance_KNNReg.pdf')
plt.show()


#ANN Model Prediction for AI Score

standardize dataset for ANN

In [ ]:
std_cols = (int_cols - int_cols.mean()) / int_cols.std()

scaler = StandardScaler()
scaled_cols = scaler.fit_transform(std_cols)
sstar_df = pd.DataFrame(scaled_cols, columns=std_cols.columns)

In [ ]:
std_cols

In [ ]:
X =std_cols.drop(columns=["total_emissns", "Emission_objtv", "emissn_score", "dirct_cemissions", "indirct_cemissions", "Grenhous_emissn", "CO2Emissn_%+-", "scope1", "scope2", "total_enrgy_use", "renwbl_enrgy_use", "envrmt_pilr_score", "Climate_incentive"])
y = std_cols["total_emissns"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size =0.3, random_state=42)

In [ ]:
model = Sequential([
    Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    Dropout(0.5),
    Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),
    Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dense(1)  # No activation function for regression
])

model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=36, validation_split=0.2, callbacks=[early_stopping])

# Evaluate model
loss = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}')

In [ ]:
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('/content/loss_plotTotlEmssn.pdf')
plt.show()

In [ ]:
y_pred = model.predict(X_test)
y_pred

In [ ]:
vy_test = y_test.values.reshape(-1, 1)
inv_ytest = scaler.inverse_transform(np.hstack((np.zeros((len(vy_test), std_cols.shape[1]-1)), vy_test)))

vy_pred = y_pred.reshape(-1, 1)
inv_ypred = scaler.inverse_transform(np.hstack((np.zeros((len(vy_pred), std_cols.shape[1]-1)), vy_pred)))

In [ ]:
y_testS = inv_ytest * int_cols['total_emissns'].std() + int_cols['total_emissns'].mean()
y_predS = inv_ypred * int_cols['total_emissns'].std() + int_cols['total_emissns'].mean()

comparison_df = pd.DataFrame({'Actual': y_testS.flatten(), 'Predicted': y_predS.flatten()})

print(comparison_df)

difference = np.abs(comparison_df['Actual'] - comparison_df['Predicted'])

cmap = plt.get_cmap('coolwarm')  # Choose a colormap
norm = plt.Normalize(difference.min(), difference.max())  # Normalize the difference values
colors = cmap(norm(difference))

plt.scatter(comparison_df['Actual'], comparison_df['Predicted'], c=colors)
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Scatterplot of Actual vs Predicted AI Score using ANN')
plt.colorbar(label='Absolute Difference')
plt.savefig('/content/scatter_plot_ANN(TotlEmssn).pdf')
plt.show()

plt.figure(figsize=(10, 5))
plt.plot(comparison_df['Actual'], label='Actual', color='blue')
plt.plot(comparison_df['Predicted'], label='Predicted', color='red')
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Lineplot of Actual vs Predicted Emission Score using ANN')
plt.legend()
plt.savefig('/content/line_plot_ANN(TotlEmssn).pdf')
plt.show()

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Calculate MAE
mae = mean_absolute_error(y_testS, y_predS)

# Calculate MSE
mse = mean_squared_error(y_testS, y_predS)

# Calculate RMSE
rmse = mean_squared_error(y_testS, y_predS, squared=False)

# Calculate R-squared
r_squared = r2_score(y_testS, y_predS)

print("Mean Absolute Error:", mae)
print("Mean Squared Error:", mse)
print("Root Mean Squared Error:", rmse)
print("R-squared:", r_squared)

In [ ]:
# Get the weights of the connections between the input layer and the first hidden layer
weights = model.layers[0].get_weights()[0]

# Calculate feature importance by summing the absolute weights for each feature
feature_importance = np.sum(np.abs(weights), axis=1)

# Get the indices of the top 50 features
top_50_indices = feature_importance.argsort()[-20:][::-1]

# Get the names of the top 50 features
top_50_features = X_train.columns[top_50_indices]

# Plot the feature importance
plt.figure(figsize=(15, 10))
plt.barh(range(len(top_50_features)), feature_importance[top_50_indices], align='center')
plt.yticks(range(len(top_50_features)), top_50_features)
plt.xlabel('Feature Importance')
plt.ylabel('Feature')
plt.title('Top 20 Features Importance for Total Emissions Using ANN')
plt.gca().invert_yaxis()
plt.savefig('/content/top_20_ANN(TotlEmssn).pdf')
plt.show()


#Random Forest Regression Analysis for AI Score

In [ ]:
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

# Feature importances
feature_importances = rf_model.feature_importances_

# Getting the top 50 features
top_50_indices = feature_importances.argsort()[-50:][::-1]
top_50_features = X.columns[top_50_indices]
top_50_importances = feature_importances[top_50_indices]

# Plotting the top 50 features
plt.figure(figsize=(20, 16))
plt.barh(top_50_features, top_50_importances)
plt.xlabel('Feature Importance')
plt.title('Top 50 Features')
plt.gca().invert_yaxis()  # Invert y-axis to display most important features at the top
plt.savefig('/content/RF_top_50_ANN.pdf')
plt.show()

# Predicting on the test set
y_pred = rf_model.predict(X_test)

# Creating a DataFrame to display actual and predicted values side by side
results_df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})
print(results_df)

#KNN Regression Analysis for AI Ranking

In [ ]:
A =std_cols.drop(columns=["total_emissns", "Emission_objtv", "emissn_score", "dirct_cemissions", "indirct_cemissions", "Grenhous_emissn", "CO2Emissn_%+-", "scope1", "scope2", "total_enrgy_use", "renwbl_enrgy_use", "envrmt_pilr_score", "Climate_incentive"])
B = std_cols["total_emissns"]

A_train, A_test, B_train, B_test = train_test_split(A, B, test_size =0.3, random_state=42)

In [ ]:
print(B.shape)

In [ ]:
# Initialize the KNN regressor
knn_regressor = KNeighborsRegressor(n_neighbors=5)  # You can adjust the number of neighbors

# Train the KNN model
knn_regressor.fit(A_train, B_train)

# Predictions
y_pred_train = knn_regressor.predict(A_train)
y_pred_test = knn_regressor.predict(A_test)

# Evaluate the model
#RMSE
train_rmse = mean_squared_error(B_train, y_pred_train, squared=False)
test_rmse = mean_squared_error(B_test, y_pred_test, squared=False)

#MAE
train_mae = mean_absolute_error(B_train, y_pred_train)
test_mae = mean_absolute_error(B_test, y_pred_test)

# R-squared
train_r2 = r2_score(B_train, y_pred_train)
test_r2 = r2_score(B_test, y_pred_test)

print(f'Train RMSE: {train_rmse}')
print(f'Test RMSE: {test_rmse}')
print(f'Train MAE: {train_mae}')
print(f'Test MAE: {test_mae}')
print(f'Train R-squared: {train_r2}')
print(f'Test R-squared: {test_r2}')


In [ ]:
compare_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})
print(compare_df)

In [ ]:
import matplotlib.pyplot as plt

# Create a DataFrame with actual and predicted rankings
comparison_df = pd.DataFrame({'Actual': B_test, 'Predicted': y_pred_test})

# Reset index to have numerical index
comparison_df.reset_index(drop=True, inplace=True)

# Plot the actual and predicted rankings
plt.figure(figsize=(14, 8))

# Bar chart
plt.subplot(2, 1, 1)  # 2 rows, 1 column, plot 1
comparison_df.plot(kind='bar', ax=plt.gca())
plt.title('Actual vs Predicted Rankings')
plt.xlabel('Sample Index')
plt.ylabel('Ranking')
plt.xticks(rotation=45)

# Scatter plot with line
plt.subplot(2, 1, 2)  # 2 rows, 1 column, plot 2
plt.scatter(B_train, y_pred_train, label='Training data', color='blue', alpha=0.5)
plt.scatter(B_test, y_pred_test, label='Test data', color='red', alpha=0.5)
plt.plot([B_train.min(), B_train.max()], [B_train.min(), B_train.max()], 'k--', lw=2)  # Diagonal line
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted values for AI Ranking Using KNN Regression')
plt.savefig('/content/top_50_KNN_Regression.pdf')
plt.legend()

plt.tight_layout()

plt.show()


In [ ]:
plt.figure(figsize=(10, 5))
plt.scatter(B_train, y_pred_train, label='Training data', color='blue', alpha=0.5)
plt.scatter(B_test, y_pred_test, label='Test data', color='red', alpha=0.5)
plt.xlabel('Actual values')
plt.ylabel('Predicted values')
plt.title('Actual vs Predicted values for KNN Regression Model')
plt.legend()
plt.show()

In [ ]:
# Calculate the absolute difference in predictions when a feature value is perturbed
train_feature_importances = np.mean(np.abs(knn_regressor.predict(A_train) - knn_regressor.predict(A_train.values + 0.01)), axis=0)
test_feature_importances = np.mean(np.abs(knn_regressor.predict(A_test) - knn_regressor.predict(A_test.values + 0.01)), axis=0)

# Get the indices of features sorted by importance
train_sorted_indices = train_feature_importances.argsort()[::-1]
test_sorted_indices = test_feature_importances.argsort()[::-1]

# Get the top 50 features and their importance scores
top_train_features = A_train.columns[train_sorted_indices][:50]
top_train_scores = train_feature_importances[train_sorted_indices][:50]

top_test_features = A_test.columns[test_sorted_indices][:50]
top_test_scores = test_feature_importances[test_sorted_indices][:50]

# Plot the top 50 features and their importance scores
plt.figure(figsize=(15, 10))

plt.subplot(2, 1, 1)
plt.barh(top_train_features, top_train_scores, color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance - Train Data')
plt.gca().invert_yaxis()  # Invert y-axis to display in descending order

plt.subplot(2, 1, 2)
plt.barh(top_test_features, top_test_scores, color='skyblue')
plt.xlabel('Importance Score')
plt.ylabel('Feature')
plt.title('Top 50 Features Importance - Test Data')
plt.gca().invert_yaxis()  # Invert y-axis to display in descending order

plt.tight_layout()
plt.show()
